##  Chameleon/FABRIC L3 Stitching

This notebook demonstrates how to stitch Chameleon experiments to FABRIC.

### Set Chameleon Environment

FABRIC should already be configured in your Jupyter environment. If this is your first time using FABRIC, may need to follow the [Configure Environment](./fabric_examples/fablib_api/configure_environment/configure_environment.ipynb) notebook to complete the configuration.

Set the following environment vars to the values found in your custom Chameleon-openrc.sh file. You can obtain your Chameleon-openrc.sh from Chameleon using these [directions](https://chameleoncloud.readthedocs.io/en/latest/technical/cli.html#the-openstack-rc-script).  The expected password is not your regular Chameleon password. Instead, you need to create a Chameleon CLI password by following these [directions](https://chameleoncloud.readthedocs.io/en/latest/technical/cli.html#cli-authentication)

In [ ]:
import os
import sys

module_path = os.path.abspath(os.path.join(f"{os.environ['HOME']}/work/PRUTH-FABRIC-Examples/fablib_local"))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from chameleon_utils.chameleon_config import *
load_chameleon_rc_environment(chameleon_rc_file=f"{os.environ['HOME']}/work/fablib_local_private_config/Chameleon-openrc.sh")

    
from fablib_custom.fablib_custom import *
from fablib_common_utils.utils import *
from fablib_common_utils.fabric_fabnet_slice import *

from chameleon_utils.chameleon_stitching import *
from chameleon_utils.chameleon_servers import *
from chameleon_utils.chameleon_leases import *


Pick a name for the Chameleon and FABRIC resources. A common issue you might encounter is using a name that is already used on Chameleon. One suggestion is to embedded the date/time in the name to increase the likelyhood that the name is unique.

In [ ]:
time_stamp = datetime.now(tz=tz.tzutc()).strftime('%Y%m%d%H%M')

site='UTAH'

slice_name=f"pruth_fabnet_{site}_{time_stamp}"

print(f'time_stamp: {time_stamp}')

## Add Nodes to FABRIC

In [ ]:
slice = create_fabnet_slice(name=slice_name, node_count=1, sites=[site], cores=4, ram=16, disk=500)

## Do something with the Nodes

In [ ]:
threads = []
for node in slice.get_nodes():
    threads.append(node.execute_thread(f'ping -c 5  10.132.1.1'))
    
for thread in threads:
    stdout, stderr = thread.result()
    print(stdout)
    print(stderr)

In [ ]:
for node in slice.get_nodes():
    
    node.upload_directory('node_utils', '.')
    
    command = 'chmod +x node_utils/*.sh && sudo ./node_utils/host_network_tune.sh'
    stdout, stderr = node.execute(command)
    print(stdout)
    print(stderr)
    
    #stdout, stderr = node.execute(f'ping -c 5 10.129.129.1')
    #print(stdout)
    #print(stderr)
    #stdout, stderr = node.execute(f'ping -c 5 10.129.129.250')
    #print(stdout)
    #print(stderr)
    #for server_name,fixed_ip in fixed_ips.items():
    #    print(f'---- {node.get_name()} -> {server_name} {fixed_ip}---')
    #    stdout, stderr = node.execute(f'ping -c 5 {fixed_ip}')    
    #    print(stdout)
    #    print(stderr)

In [ ]:
private_key='/home/fabric/work/fablib_local_private_config/my_chameleon_key'
server_ip = '10.129.129.23'


for node in slice.get_nodes():

    #for file_name in [ 'file.100G', 'file.10G', 'file.25G', 'file.50G', 'file.75G']:
    for file_name in [ 'file.50G' ]:
        command = f'curl -o {file_name} http://{server_ip}/{file_name}'

        print(command)
        stdout, stderr = node.execute(command)
        print(stdout)
        print(stderr)

# Clean Up 

In [ ]:
try:
    slice.delete()

    #fablib.show_config()
    #fablib.delete_all()
except Exception as e:
    print(f"Exception: {e}") 

In [ ]:
try:
    fablib = fablib_manager()

    #fablib.show_config()
    #fablib.delete_all()
except Exception as e:
    print(f"Exception: {e}") 